In [9]:
# NOTE: Must have run the "Estimate RAM Size of Enemies" example for this to run
import os,json
if not os.path.isdir("workspace"):
    os.mkdir("workspace")
    
locs = json.load(open("locations.json"))
records = json.load(open(os.path.join("..", "Estimate RAM Size of Enemies", "workspace", "enemy_records_filled.json")))

In [10]:
# Each room is ard (and associated object files for each object)
#    +msn
#    +dbg world files
#    +event effect files 
#    +map file
#    +msg file
#    +voice effects
BASEPATH = os.path.join(os.environ["USE_KH2_GITPATH"], "KH2")
WORLDMAP = {
    '00': 'ZZ',
    '01': 'ES',
    '02': 'TT',
    '03': 'DI',
    '04': 'HB',
    '05': 'BB',
    '06': 'HE',
    '07': 'AL',
    '08': 'MU',
    '09': 'PO',
    '0A': 'LK',
    '0B': 'LM',
    '0C': 'DC',
    '0D': 'WI',
    '0E': 'NM',
    '0F': 'WM',
    '10': 'CA',
    '11': 'TR',
    '12': 'EH'
}
def lookupEnemySize(enemy):
    if len(enemy) == 4:
        aiparam = ''
    if len(enemy) == 6:
        aiparam = enemy[1]
        enemy = enemy[2:]
    for e in records.values():
        if e["ucm"].lower().zfill(4) == enemy.lower():
            if e["aiparam"] == '0' or e["aiparam"] == '':
                if aiparam == '0' or aiparam == '':
                    return e["size"]
            if e["aiparam"] == aiparam:
                return e["size"]
    raise Exception("enemy not found {}".format(enemy))
def lookupEnemyName(enemy):
    if len(enemy) == 4:
        aiparam = ''
    if len(enemy) == 6:
        aiparam = enemy[1]
        enemy = enemy[2:]
    for e in records.values():
        if e["ucm"].lower().zfill(4) == enemy.lower():
            if e["aiparam"] == '0' or e["aiparam"] == '':
                if aiparam == '0' or aiparam == '':
                    return e["name"]
            if e["aiparam"] == aiparam:
                return e["name"]
    raise Exception("enemy not found {}".format(enemy))
with open("msnlocator.csv") as f:
    m = f.read().split("\n")
    msns = {}
    for xx in m[1:]:
        if len(xx) == 0:
            continue
        xxx = xx.split(",")
        msns[xxx[0].strip()] = {"msn": xxx[2].strip()}
        if len(xxx) >= 4 and len(xxx[3]) > 0:
            msns[xxx[0].strip()]["over_ard"] = xxx[3]
        if len(xxx) >= 5 and len(xxx[4]) > 0:
            msns[xxx[0].strip()]["over_map"] = xxx[4]
for loc in locs:
    # translate world later
    world = WORLDMAP[loc["world"]]
    room = str(int(loc["room"],16)).zfill(2)
    event = str(int(loc["event"], 16)).zfill(2)
    msnname = msns[loc["description"]]["msn"] + ".bar"
    ardname = msns[loc["description"]]["over_ard"] if "over_ard" in msns[loc["description"]] else world.lower()+room+".ard"
    mapname = msns[loc["description"]]["over_map"] if "over_ard" in msns[loc["description"]] else world.lower()+room+".map"
    ARDFN = os.path.join(BASEPATH, "ard", ardname)
    # msn - msn/jp WORLDROOM_MSEVENT.bar
    ## Skipping MSN files for now
    MSNFN = os.path.join(BASEPATH, "msn", "jp", msnname)
    # dbg/ localset-WORLD.bin
    LOCALSETFN = os.path.join(BASEPATH, "dbg", "localset-"+world.lower()+".bin")
    # dbg/ mission-WORLD.dbg
    MISSIONFN = os.path.join(BASEPATH, "dbg", "mission-"+world.lower()+".dbg")
    # dbg/ progress-WORLD.dbg
    PROGRESSFN = os.path.join(BASEPATH,"dbg","progress-"+world.lower()+".dbg")
    # event effect files - idk how to tell
    # map/jp WORLDROOM.map
    MAPFN = os.path.join(BASEPATH, "map", "jp", mapname)
    # msg/jp WORLD.bar
    MSGFN = os.path.join(BASEPATH, "msg", "jp", world.lower()+".bar")
    # voice looks complicated to parse and not much there
    files = [ARDFN,LOCALSETFN,MSNFN,MISSIONFN,PROGRESSFN,MAPFN,MSGFN]
    size = 0
    for f in files:
        if not os.path.exists(f):
            print("Warning - {} - can't find {}".format(loc["description"], f))
        else:
            size += os.path.getsize(f)
    loc["size"] = size
    size_inc_enemies = size
    for enemy in loc["enemies"]:
        size_inc_enemies += lookupEnemySize(enemy["enemy"]["value"])
        enemy["enemy"]["name"] = lookupEnemyName(enemy["enemy"]["value"])
    loc["size_inc_enemies"] = size_inc_enemies
json.dump(locs, open(os.path.join("workspace", "locations_with_size.json"), "w"), sort_keys=True, indent=4, separators=(',', ': '))

In [11]:
c = "descr,ard\n"
for r in sorted(locs, key=lambda k: k["size_inc_enemies"]):
    world = WORLDMAP[r["world"]]
    room = str(int(r["room"],16)).zfill(2)
    ard = world.lower()+room+".ard"
    c += "{},{}\n".format(r["description"], ard)
    print("{} {}".format(r["size_inc_enemies"], r["description"]))

2863574 The Expotition
2997056 Armor Xemnas II Boss Fight
3031644 Scar Boss Fight
3569712 Hostile Program Boss Fight
3685882 The World of Nothing (Cylinder Lasers) Fight
3725606 Hades Paradox Round 10
3726198 Lexaeus AS Fight
3726198 Lexaeus Data Fight
3753048 Armor Xemnas I Boss Fight
3863468 Storm Rider Boss Fight
3945332 Hades Chamber Boss Fight
4011032 Pete (Past) Boss Fight
4184526 Marluxia AS Fight
4184526 Marluxia Data Fight
4202626 Gambler (Ship Graveyard: Seadrift Keep; 1st Fight)
4244178 Hades Boss Phase II
4314254 Hades Paradox Round 25
4319428 Sark Boss Fight
4448568 Lingering Will Boss Fight
4478464 Grim Reaper I Boss Fight
4507606 Prison Keeper Boss Fight
4526214 Station of Serenity Nobodies
4575808 Axel II Boss Fight
4576884 Axel Data Fight
4590810 Dark Thorn Boss Fight
4705796 Pain/Panic Round 10
4762392 Present Collecting Fight
4837582 Xigbar Data Fight
4951984 Titan Round 10
5003784 Cerberus Boss Fight
5013802 Dusk (1st Fight with Keyblade)
5203738 Hades Cup - Semifin